# Symbolic computation

In [9]:
import sympy as sym
import sympy.printing as printing

In [68]:
theta, phi, lamb, omega = sym.symbols('Theta Phi Lambda Omega')
alpha, gamma = sym.symbols("alpha gamma")
torque_par, torque_per = sym.symbols("tau_1 tau_2")
K1, K2 = sym.symbols("K1 K2")
Ms, Hoe = sym.symbols("M_s H_{oe}")
# energy = sym.Function("U")(theta, phi, lamb, omega)

In [74]:
"""
Theta, azimuth L1
Phi, inplane L1

Lambda, azimuth L1
Omega, inplane L2
"""

sinT = sym.sin(theta)
cosT = sym.cos(theta)

cosL = sym.cos(lamb)
sinL = sym.sin(lamb)

sinO = sym.sin(omega)
cosO = sym.cos(omega)

sinP = sym.sin(phi)
cosP = sym.cos(phi)

cosPO = sym.cos(phi - omega)
sinPO = sym.sin(phi - omega)

const = (gamma / Ms) * (1 / (1 + alpha**2))

Ms1, Ms2 = sym.symbols("M_{s1} M_{s2}")
He1, He2, He3 = sym.symbols("H_{ext1} H_{ext2} H_{ext3}")
Hd11, Hd12, Hd13 = sym.symbols("H_{dip11} H_{dip12} H_{dip13}")
Hd21, Hd22, Hd23 = sym.symbols("H_{dip21} H_{dip22} H_{dip23}")
Hde11, Hde12, Hde13 = sym.symbols("H_{de11} H_{de12} H_{de13}")
Hde21, Hde22, Hde23 = sym.symbols("H_{de21} H_{de22} H_{de23}")

Hext = sym.Matrix([He1, He2, He3])
Hdip1 = sym.Matrix([Hd11, Hd12, Hd13])
Hdip2 = sym.Matrix([Hd21, Hd22, Hd23])
Hdemag1 = sym.Matrix([Hde11, Hde12, Hde13])
Hdemag2 = sym.Matrix([Hde21, Hde22, Hde23])

M1 = sym.Matrix([Ms1 * sinT * cosP, Ms1 * sinT * sinP, Ms1 * cosT])
M2 = sym.Matrix([Ms2 * sinL * cosO, Ms2 * sinL * sinO, Ms2 * cosL])

anisotropies = K1 * (cosT**2 + (sinT**2) *
                     (sinP**2)) + K2 * (cosL**2 + (sinL**2) * (sinO**2))
dipole = -M1.dot(Hdip2) - M2.dot(Hdip1)
demag = -M1.dot(Hdemag1) - M2.dot(Hdemag2)
ext = -M1.dot(Hext) - M2.dot(Hext)

energy = anisotropies + dipole + demag + ext
dUdP = sym.diff(energy, phi)
dUdT = sym.diff(energy, theta)
dUdO = sym.diff(energy, omega)
dUdL = sym.diff(energy, lamb)

In [75]:
energy

-H_{de11}*M_{s1}*sin(Theta)*cos(Phi) - H_{de12}*M_{s1}*sin(Phi)*sin(Theta) - H_{de13}*M_{s1}*cos(Theta) - H_{de21}*M_{s2}*sin(Lambda)*cos(Omega) - H_{de22}*M_{s2}*sin(Lambda)*sin(Omega) - H_{de23}*M_{s2}*cos(Lambda) - H_{dip11}*M_{s2}*sin(Lambda)*cos(Omega) - H_{dip12}*M_{s2}*sin(Lambda)*sin(Omega) - H_{dip13}*M_{s2}*cos(Lambda) - H_{dip21}*M_{s1}*sin(Theta)*cos(Phi) - H_{dip22}*M_{s1}*sin(Phi)*sin(Theta) - H_{dip23}*M_{s1}*cos(Theta) - H_{ext1}*M_{s1}*sin(Theta)*cos(Phi) - H_{ext1}*M_{s2}*sin(Lambda)*cos(Omega) - H_{ext2}*M_{s1}*sin(Phi)*sin(Theta) - H_{ext2}*M_{s2}*sin(Lambda)*sin(Omega) - H_{ext3}*M_{s1}*cos(Theta) - H_{ext3}*M_{s2}*cos(Lambda) + K1*(sin(Phi)**2*sin(Theta)**2 + cos(Theta)**2) + K2*(sin(Lambda)**2*sin(Omega)**2 + cos(Lambda)**2)

In [21]:
dTheta = const * ((-1 / sinT)*dUdP - alpha*dUdT -
                  (cosL * sinT - cosT * sinL * cosPO) *
                  (torque_par + alpha * torque_per) - (sinL * sinPO) *
                  (alpha * torque_par - torque_per))

In [22]:
dTheta

gamma*(-K1*alpha*(2*sin(Phi)**2*sin(Theta)*cos(Theta) - 2*sin(Theta)*cos(Theta)) - 2*K1*sin(Phi)*sin(Theta)*cos(Phi) + (alpha*tau_1 - tau_2)*sin(Lambda)*sin(Omega - Phi) - (alpha*tau_2 + tau_1)*(-sin(Lambda)*cos(Theta)*cos(Omega - Phi) + sin(Theta)*cos(Lambda)))/(M_s*(alpha**2 + 1))

In [23]:
sym.diff(dTheta, omega)

gamma*((alpha*tau_1 - tau_2)*sin(Lambda)*cos(Omega - Phi) + (-alpha*tau_2 - tau_1)*sin(Lambda)*sin(Omega - Phi)*cos(Theta))/(M_s*(alpha**2 + 1))

In [24]:
dPhi = const * ((1 / sinT) * dUdT - (alpha / (sinT**2)) * dUdP + (1 / sinT) *
                (cosL * sinT - cosT * sinL * cosPO) *
                (alpha * torque_par - torque_per) 
                - (1/sinT)*(sinL*sinPO)*(torque_par + alpha*torque_per) + alpha*Ms*Hoe/sinT)
dPhi

gamma*(H_{oe}*M_s*alpha/sin(Theta) - 2*K1*alpha*sin(Phi)*cos(Phi) + K1*(2*sin(Phi)**2*sin(Theta)*cos(Theta) - 2*sin(Theta)*cos(Theta))/sin(Theta) + (alpha*tau_1 - tau_2)*(-sin(Lambda)*cos(Theta)*cos(Omega - Phi) + sin(Theta)*cos(Lambda))/sin(Theta) + (alpha*tau_2 + tau_1)*sin(Lambda)*sin(Omega - Phi)/sin(Theta))/(M_s*(alpha**2 + 1))

In [25]:
dLam = const * ((-1 / sinT)*dUdO - alpha*dUdL +
                  (-sinL * cosT + sinT * cosL * cosPO) *
                  (torque_par + alpha * torque_per) + (sinL * sinPO) *
                  (alpha * torque_par + torque_per) + Ms*Hoe) 
dLam

gamma*(H_{oe}*M_s - K2*alpha*(2*sin(Lambda)*sin(Omega)**2*cos(Lambda) - 2*sin(Lambda)*cos(Lambda)) - 2*K2*sin(Lambda)**2*sin(Omega)*cos(Omega)/sin(Theta) - (alpha*tau_1 + tau_2)*sin(Lambda)*sin(Omega - Phi) + (alpha*tau_2 + tau_1)*(-sin(Lambda)*cos(Theta) + sin(Theta)*cos(Lambda)*cos(Omega - Phi)))/(M_s*(alpha**2 + 1))

In [26]:
dOmega = const * ((1 / sinT) * dUdL - (alpha / (sinL**2)) * dUdO - (1 / sinL) *
                (sinL * cosT - sinT *cosL * cosPO) *
                (torque_per - alpha * torque_par) 
                + (1/sinL)*(sinT*sinPO)*(torque_par - alpha*torque_per) + alpha*Ms*Hoe/sinL)
dOmega

gamma*(H_{oe}*M_s*alpha/sin(Lambda) - 2*K2*alpha*sin(Omega)*cos(Omega) + K2*(2*sin(Lambda)*sin(Omega)**2*cos(Lambda) - 2*sin(Lambda)*cos(Lambda))/sin(Theta) - (-alpha*tau_1 + tau_2)*(sin(Lambda)*cos(Theta) - sin(Theta)*cos(Lambda)*cos(Omega - Phi))/sin(Lambda) - (-alpha*tau_2 + tau_1)*sin(Theta)*sin(Omega - Phi)/sin(Lambda))/(M_s*(alpha**2 + 1))

In [27]:
dM = sym.Matrix([
    [
        sym.diff(dTheta, theta),
        sym.diff(dTheta, phi),
        sym.diff(dTheta, lamb),
        sym.diff(dTheta, omega)
    ],
    [
        sym.diff(dPhi, theta),
        sym.diff(dPhi, phi),
        sym.diff(dPhi, lamb),
        sym.diff(dPhi, omega)
    ],
    [
        sym.diff(dLam, theta),
        sym.diff(dLam, phi),
        sym.diff(dLam, lamb),
        sym.diff(dLam, omega)
    ],
    [
        sym.diff(dOmega, theta),
        sym.diff(dOmega, phi),
        sym.diff(dOmega, lamb),
        sym.diff(dOmega, omega)
    ]]
)

In [28]:
dM

Matrix([
[                                                                                                                                                                                                                                                                                                                        gamma*(-K1*alpha*(-2*sin(Phi)**2*sin(Theta)**2 + 2*sin(Phi)**2*cos(Theta)**2 + 2*sin(Theta)**2 - 2*cos(Theta)**2) - 2*K1*sin(Phi)*cos(Phi)*cos(Theta) + (-alpha*tau_2 - tau_1)*(sin(Lambda)*sin(Theta)*cos(Omega - Phi) + cos(Lambda)*cos(Theta)))/(M_s*(alpha**2 + 1)), gamma*(-4*K1*alpha*sin(Phi)*sin(Theta)*cos(Phi)*cos(Theta) + 2*K1*sin(Phi)**2*sin(Theta) - 2*K1*sin(Theta)*cos(Phi)**2 - (alpha*tau_1 - tau_2)*sin(Lambda)*cos(Omega - Phi) - (-alpha*tau_2 - tau_1)*sin(Lambda)*sin(Omega - Phi)*cos(Theta))/(M_s*(alpha**2 + 1)),                                                                                                                                                          

In [30]:
# dM.eigenvals()

# Solver

In [7]:
import numpy as np
M = np.array([0.2, 0.5, 0.6])
Hext = np.array([120, 0, 2])
N = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
M*Hext, N@M, M@N

(array([24. ,  0. ,  1.2]), array([0.7, 0.5, 0.6]), array([0.2, 0.7, 0.6]))

In [76]:
dUdP

H_{de11}*M_{s1}*sin(Phi)*sin(Theta) - H_{de12}*M_{s1}*sin(Theta)*cos(Phi) + H_{dip21}*M_{s1}*sin(Phi)*sin(Theta) - H_{dip22}*M_{s1}*sin(Theta)*cos(Phi) + H_{ext1}*M_{s1}*sin(Phi)*sin(Theta) - H_{ext2}*M_{s1}*sin(Theta)*cos(Phi) + 2*K1*sin(Phi)*sin(Theta)**2*cos(Phi)

In [78]:
energy

-H_{de11}*M_{s1}*sin(Theta)*cos(Phi) - H_{de12}*M_{s1}*sin(Phi)*sin(Theta) - H_{de13}*M_{s1}*cos(Theta) - H_{de21}*M_{s2}*sin(Lambda)*cos(Omega) - H_{de22}*M_{s2}*sin(Lambda)*sin(Omega) - H_{de23}*M_{s2}*cos(Lambda) - H_{dip11}*M_{s2}*sin(Lambda)*cos(Omega) - H_{dip12}*M_{s2}*sin(Lambda)*sin(Omega) - H_{dip13}*M_{s2}*cos(Lambda) - H_{dip21}*M_{s1}*sin(Theta)*cos(Phi) - H_{dip22}*M_{s1}*sin(Phi)*sin(Theta) - H_{dip23}*M_{s1}*cos(Theta) - H_{ext1}*M_{s1}*sin(Theta)*cos(Phi) - H_{ext1}*M_{s2}*sin(Lambda)*cos(Omega) - H_{ext2}*M_{s1}*sin(Phi)*sin(Theta) - H_{ext2}*M_{s2}*sin(Lambda)*sin(Omega) - H_{ext3}*M_{s1}*cos(Theta) - H_{ext3}*M_{s2}*cos(Lambda) + K1*(sin(Phi)**2*sin(Theta)**2 + cos(Theta)**2) + K2*(sin(Lambda)**2*sin(Omega)**2 + cos(Lambda)**2)

In [77]:
dUdT

-H_{de11}*M_{s1}*cos(Phi)*cos(Theta) - H_{de12}*M_{s1}*sin(Phi)*cos(Theta) + H_{de13}*M_{s1}*sin(Theta) - H_{dip21}*M_{s1}*cos(Phi)*cos(Theta) - H_{dip22}*M_{s1}*sin(Phi)*cos(Theta) + H_{dip23}*M_{s1}*sin(Theta) - H_{ext1}*M_{s1}*cos(Phi)*cos(Theta) - H_{ext2}*M_{s1}*sin(Phi)*cos(Theta) + H_{ext3}*M_{s1}*sin(Theta) + K1*(2*sin(Phi)**2*sin(Theta)*cos(Theta) - 2*sin(Theta)*cos(Theta))

In [106]:
import numpy as np
from copy import deepcopy

TtoAm = 795774.715459
"""
        Spherical coords
        Theta -- azimuth angle, L1
        Phi -- inplane angle, L1
        Omega -- inplane angle, L2
        Lambda -- azimuth angle, L2
"""


def energy_cost_fn(theta, phi, lambda_, omega, Ms1, Ms2, K1, K2, Hext, Hdemag1,
                   Hdemag2, Hdipole1, Hdipole2):

    sT = np.sin(theta)
    cT = np.cos(theta)

    sP = np.sin(phi)
    cP = np.cos(phi)

    sL = np.sin(lambda_)
    cL = np.cos(lambda_)

    sO = np.sin(omega)
    cO = np.cos(omega)

    TP = np.array([sT * cP, sP * sT, cT])

    OL = np.array([sL * cO, sL * sO, cL])

    A1 = K1 * (np.power(cT, 2) + np.power(sT, 2) * np.power(sP, 2))
    A2 = K2 * (np.power(cL, 2) + np.power(sL, 2) * np.power(sO, 2))

    cost = A1 + A2
    for H, M, A in zip([Hdemag1, Hdemag2, Hdipole1, Hdipole2, Hext, Hext],
                       [Ms1, Ms2, Ms2, Ms1, Ms1, Ms2],
                       [TP, OL, OL, TP, TP, OL]):
        cost -= np.sum(H * M * A)

    return cost


def gradient_vector_U(Ms, K, Hext, Hdemag, Hdipole, inplane_angle,
                      azimuth_angle):
    """
    [inplane, azimuth],
    [phi, theta],
    [omega, lambda]
    """
    K_gradient = np.array([
        2 * K * np.sin(azimuth_angle) * np.power(np.sin(inplane_angle), 2) *
        np.cos(inplane_angle), 2 * K * np.sin(inplane_angle) *
        np.cos(inplane_angle) * (np.power(np.sin(azimuth_angle), 2) - 1)
    ])

    # virtually the same types of interaction -- dipole & demag
    Hdemag_gradient = np.array([
        Hdemag[0] * np.sin(inplane_angle) * np.sin(azimuth_angle) -
        Hdemag[1] * np.sin(azimuth_angle) * np.cos(inplane_angle),
        -Hdemag[0] * np.cos(inplane_angle) * np.cos(azimuth_angle) -
        Hdemag[1] * np.sin(inplane_angle) * np.cos(azimuth_angle) +
        Hdemag[2] * np.sin(azimuth_angle)
    ]) * Ms

    Hdip_gradient = np.array([
        Hdipole[0] * np.sin(inplane_angle) * np.sin(azimuth_angle) -
        Hdipole[1] * np.sin(azimuth_angle) * np.cos(inplane_angle),
        -Hdipole[0] * np.cos(inplane_angle) * np.cos(azimuth_angle) -
        Hdipole[1] * np.sin(inplane_angle) * np.cos(azimuth_angle) +
        Hdipole[2] * np.sin(azimuth_angle)
    ]) * Ms

    Hext_gradient = np.array([
        Hext[0] * np.sin(inplane_angle) * np.sin(azimuth_angle) -
        Hext[1] * np.sin(azimuth_angle) * np.cos(inplane_angle),
        -Hext[0] * np.cos(inplane_angle) * np.cos(azimuth_angle) -
        Hext[1] * np.sin(inplane_angle) * np.cos(azimuth_angle) +
        Hext[2] * np.sin(azimuth_angle)
    ]) * Ms

    return K_gradient + Hdemag_gradient + Hdip_gradient + Hext_gradient


def gradientDescent(param_set,
                    Ndemag,
                    Ndipole,
                    layer='A',
                    num_iterations=10000,
                    lr=0.01, w=10):
    """
    x = [inplane, azimuth]
    """
    if layer == 'A':
        Ms, K, Hext = param_set['Ms1'], param_set["K1"], param_set["Hext"]
        Hdipole, Hdemag, = param_set["Hdipole2"], param_set["Hdemag1"]
        x = np.array([param_set['phi'], param_set['theta']])
    elif layer == "B":
        Ms, K, Hext = param_set['Ms2'], param_set["K2"], param_set["Hext"]
        Hdipole, Hdemag, = param_set["Hdipole1"], param_set["Hdemag2"]
        x = np.array([param_set['omega'], param_set['lambda_']])
    else:
        raise ValueError("Unidentified layer")

    param_set_copy = deepcopy(param_set)
    last_cost = 0
    for i in range(num_iterations):
        grad = gradient_vector_U(Ms, K, Hext, Hdemag, Hdipole, x[0], x[1])
        x = x - lr * grad
        if i % w == 0:
            if layer == 'A':
                param_set_copy['phi'] = x[0]
                param_set_copy['theta'] = x[1]

                Ms1v = cartiesian_from_spherical(Ms1, x[0], x[1])
                param_set_copy['Hdemag1'] = get_tensor_interaction(
                    Ms1v, Ndemag)
                param_set_copy['Hdipole1'] = get_tensor_interaction(
                    Ms1v, Ndipole)
            else:
                param_set_copy['omega'] = x[0]
                param_set_copy['lambda'] = x[1]

                Ms2v = cartiesian_from_spherical(Ms2, x[0], x[1])
                param_set_copy['Hdemag1'] = get_tensor_interaction(
                    Ms1v, Ndemag)
                param_set_copy['Hdipole1'] = get_tensor_interaction(
                    Ms1v, Ndipole)

            cost = energy_cost_fn(**param_set_copy)
            print(f"[{i}] Cost: {cost} -- diff {last_cost-cost}")
            last_cost = cost
    return x

In [107]:
Ndemag = np.array([[6.8353909454237e-4, 0., 0.], [0., 0.00150694452305927, 0.],
                   [0., 0., 0.99780951638608]])
Ndipole = np.array([[5.57049776248663e-4, 0., 0.],
                    [0., 0.00125355500286346, 0.],
                    [0., 0.0, -0.00181060482770131]])

def get_tensor_interaction(M, N):
    return N @ M


def cartiesian_from_spherical(r, inplane, azimuth):
    return np.array([
        r*np.sin(azimuth)*np.cos(inplane),
        r*np.sin(azimuth)*np.sin(inplane),
        r*np.cos(azimuth)
    ])
    

phi = to_radian(20) # inplane
theta = to_radian(80)    

omega = to_radian(20) # inplane
lambda_ = to_radian(80)

Ms1 = Ms2 = 1 * TtoAm
Ms1v = cartiesian_from_spherical(Ms1, phi, theta)
Ms2v = cartiesian_from_spherical(Ms2, omega, lambda_)

Hdipole1 = get_tensor_interaction(Ms1v, Ndipole)
Hdipole2 = get_tensor_interaction(Ms2v, Ndipole)

param_set = {
    'theta': theta,
    'omega': omega,
    'phi': phi,
    'lambda_': lambda_,
    'Hext': np.array([0, 1, 1]) * 0.3 * TtoAm,
    'K1': 650e3,
    'K2': 880e3,
    'Ms1': Ms1,
    'Ms2': Ms2,
    'Hdipole1': Hdipole1,
    'Hdipole2': Hdipole2,
    'Hdemag1': get_tensor_interaction(Ms1v, Ndemag),
    'Hdemag2': get_tensor_interaction(Ms2v, Ndemag)
}

gradientDescent(param_set, Ndemag, Ndipole, layer='A', num_iterations=1000, lr=0.1)

[0] Cost: -799409885480.328 -- diff 799409885480.328
[10] Cost: -939059550936.7723 -- diff 139649665456.44434
[20] Cost: -284901139695.89246 -- diff -654158411240.8799
[30] Cost: -283156677148.6479 -- diff -1744462547.2445679
[40] Cost: -555378275491.0128 -- diff 272221598342.36493
[50] Cost: -935736463239.7626 -- diff 380358187748.74976
[60] Cost: -199485846848.571 -- diff -736250616391.1915
[70] Cost: 79640802720.3289 -- diff -279126649568.8999
[80] Cost: 11838923007.691162 -- diff 67801879712.63774
[90] Cost: -500378880912.87256 -- diff 512217803920.5637
[100] Cost: -910694084853.0492 -- diff 410315203940.17664
[110] Cost: -416650553236.2378 -- diff -494043531616.8114
[120] Cost: -77486657505.28111 -- diff -339163895730.95667
[130] Cost: -829406769353.4814 -- diff 751920111848.2003
[140] Cost: -218473900518.67316 -- diff -610932868834.8083
[150] Cost: -227473636382.66074 -- diff 8999735863.98758
[160] Cost: -280395458145.40875 -- diff 52921821762.74802
[170] Cost: -565427237037.6782

array([-6.63813664e+10,  7.27730409e+10])

In [41]:




def to_radian(angle):
    return angle * (np.pi / 180)


def get_anisotropy_density(K, angle1, angle2):

    return K * (np.power(np.cos(angle1), 2) +
                np.power(np.sin(angle1), 2) * np.power(np.sin(angle2), 2))


def get_tensor_interaction(M, N):
    return N @ M


def magnetic_energy_density_cost(theta, omega, phi, lambda_, Ms1, Ms2, K1, K2)

def magnetic_energy_density(theta, omega, phi, lambda_, M1, M2, Hext, K1, K2, Ndip1, Ndip2,
                            Ndemag1, Ndemag2):
    """
    @param start params:
    is a dict of below
        Spherical coords
        Theta -- azimuth angle, L1
        Phi -- inplane angle, L1
        Omega -- inplane angle, L2
        Lambda -- azimuth angle, L2
    """

    A1 = get_anisotropy_density(K1, theta, phi)
    A2 = get_anisotropy_density(K2, lambda_, omega)
    Hdip1 = get_tensor_interaction(M1, Ndip1)
    Hdip2 = get_tensor_interaction(M2, Ndip2)
    Hdemag1 = get_tensor_interaction(M1, Ndemag1)
    Hdemag2 = get_tensor_interaction(M2, Ndemag2)
    U = A1 + A2 - M1 * Hext - M2 * Hext - M1 * Hdemag1 - M2 * Hdemag2 - M1 * Hdip1 - M2 * Hdip2
    """
    Get the minimum of the angles
    """
    return U




M1 = M2 = np.array([0, 0, 1 * TtoAm])
Hext = 0.4 * TtoAm  # 0.4 T
start_params = {

}
K1 = 650e3
K2 = 880e3
magnetic_energy_density(**params)

array([ 1.43712380e+06,  1.43712380e+06, -1.76805184e+12])